# Server Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [1]:
import os
import time

from datetime import date, datetime, timedelta

import csv
import json

import logging
import threading
import queue

import socket
from urllib.request import urlopen

from common import Printable, projdir
from threadpool import WorkerThread, WorkerThreadPool

from constants import *

## Default Settings

Default values, just in case of missing details in the config file

In [2]:
# Default threshold for Motion filesystem warning
DEFAULT_FILESYSTEM_THRESHOLD = 95

# Default threshold for Motion battery warning
DEFAULT_BATTERY_THRESHOLD = 60

## Detection Thread Pool

Detect new motions appearing on the network

In [3]:
class DetectionThread(WorkerThread):
    """Thread pool worker to detect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""

        if motion.status <= 0:
            logging.debug("%s is processing %s", self.name, motion.hostname)
        
            addrInfo = motion.getAddrInfo()
            
            if addrInfo:
                logging.debug("%s detected at %s", motion.hostname, addrInfo[0])

                status = motion.getStatus(timeout=self.timeout)
                
                if status == HOST_STATUS_CONNECTED:
                    logging.info("%s detected and is accepting connections", motion.hostname)
        
                    self.outQueue.put(motion)

                else:
                    logging.debug("%s detected but is not accepting connections - status %i", motion.hostname, status)
            
            else:
                logging.debug("%s not detected (address information unavailable)", motion.hostname)

In [4]:
class DetectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DetectionThread)
        

    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s is queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status <= 0:
                logging.debug("%s is queuing %s", self.name, motion.hostname)
                self.inQueue.put(motion)
                count += 1
                
        return count

## Motion Inspection Thread

Inspect new motions appearing on the network

In [5]:
class InspectionThread(WorkerThread):
    """Thread pool worker to inspect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual inspection of a single motion"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)

        self.motion = motion

        motion.info = self.fetchJson(motion, 'info.json')
        self.checkMotionInfo(motion)
        
        motion.settings = self.fetchJson(motion, 'settings.json')
        self.checkMotionSettings(motion)
        
        motion.logs = self.fetchJson(motion, 'logs.json')

        # Change to HOST_STATUS_INSPECTED when downloads are implemented
        motion.status = HOST_STATUS_COMPLETED
        
        logging.debug("%s successfully processed %s", self.name, motion.hostname)

        #self.outQueue.put(motion)


    def fetchJson(self, motion, filename):
        """Do the actual inspection of a single motion"""

        # TODO - write to cache files
        try:
            result = motion.fetchJson(filename, timeout=self.timeout)
        except Exception as e:
            logging.error("%s was unable to provide %s", motion.hostname, filename)
            raise
            
        return result
            

    def checkMotionInfo(self, motion):
        """Check Motion information such as firmware, filesystem and battery"""
        
        self.checkMotionFirmware(motion)
        self.checkMotionStorage(motion)
        self.checkMotionBattery(motion)

            
    def checkMotionFirmware(self, motion):
        """Check firmware is acceptable"""
        
        try:
            # 3144 onwards introduced code, name and special
            firmware = motion.info['motion']['firmware']['code']
        except Exception:
            # Prior to 3144 it was just the code
            firmware = motion.info['motion']['firmware']
        
        # Only specific firmwares are acceptable
        try: 
            firmwares = self.config['Motion']['Info']['Firmwares']
        except Exception:
            firmwares = {}

        if str(firmware) not in firmwares:
            logging.warning("%s firmware is not approved (%s)", motion.hostname, firmware)
        else:
            logging.debug("%s firmware is approved (%s)", motion.hostname, firmware)


    def checkMotionStorage(self, motion):
        """Check storage is not above acceptable threshold"""
        
        try:
            level = motion.info['motion']['filesystem']['level']
        except Exception:
            logging.warning("%s filesystem level (% storage used) was undetermined", motion.hostname)
            level = 0        
            
        try: 
            threshold = self.config['Motion']['Info']['Thresholds']['Storage']
        except Exception:
            threshold = DEFAULT_FILESYSTEM_THRESHOLD

        if int(level) > threshold:
            logging.warning("%s storage is %s%% full", motion.hostname, level)
        else:
            logging.debug("%s storage is %s%% full", motion.hostname, level)


    def checkMotionBattery(self, motion):
        """Check battery is not below acceptable threshold"""
        
        try:
            level = motion.info['motion']['battery']['level']
        except Exception:
            logging.warning("%s battery level undetermined", motion.hostname)
            level = 0        
            
        try: 
            threshold = self.config['Motion']['Info']['Thresholds']['Battery']
        except Exception:
            threshold = DEFAULT_BATTERY_THRESHOLD

        if int(level) < threshold:
            logging.warning("%s battery level is %s%%", motion.hostname, level)
        else:
            logging.info("%s battery level is %s%%", motion.hostname, level)


    def checkMotionSettings(self, motion):
        """Check Motion settings such as firmware, filesystem and battery"""

        self.checkMotionSetting(motion, 'Speed', 'speed_unit')
        self.checkMotionSetting(motion, 'Distance', 'distance_unit')
        self.checkMotionSetting(motion, 'GNSS Rate', 'gnss_rate')
        self.checkMotionSetting(motion, 'One Log Per Day', 'one_log_per_day')
        self.checkMotionSetting(motion, 'Timezone', 'timezone')

        # TODO
        #self.checkMotionFlash('Flash', 'distance_unit')


    def checkMotionSetting(self, motion, configSetting, motionSetting):
        """Check Motion setting is correct"""

        try:
            expected = self.config['Motion']['Settings'][configSetting]
            actual = motion.settings['settings'][motionSetting]

            if actual != str(expected):
                logging.warning("%s setting for %s is %s", motion.hostname, configSetting, actual)
            else:
                logging.debug("%s setting for %s is %s", motion.hostname, configSetting, actual)

        except Exception:
            logging.warning("%s setting for %s undetermined", motion.hostname, configSetting, motionSetting)

In [6]:
class InspectionThreadPool(WorkerThreadPool):
    """Thread pool manager to inspect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, InspectionThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Download Thread Pool

Download new tracks for Motions

In [7]:
class DownloadThread(WorkerThread):
    """Thread pool worker to download tracks from a motion"""

    def process(self, motion):
        """Do the actual downloads for a single motion"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)
        
        self.outQueue.put(motion)

In [8]:
class DownloadThreadPool(WorkerThreadPool):
    """Thread pool manager to download tracks from motions"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DownloadThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Disconnection Thread Pool

Detect motions disconnecting from the network

In [9]:
class DisconnectionThread(WorkerThread):
    """Thread pool worker to detect a motion disconnected from the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)
        
        status = motion.getStatus(timeout=self.timeout)

        if motion.status > 0:
            logging.debug("%s is still connected", motion.hostname)
            
            # Note that motion.getStatus() will have nuked the status, thus need to set back to HOST_STATUS_COMPLETED
            motion.status = HOST_STATUS_COMPLETED

        else:
            logging.info("%s has been disconnected", motion.hostname)

In [10]:
class DisconnectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions disconnected from the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DisconnectionThread)


    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status == HOST_STATUS_COMPLETED:
                logging.debug("%s is queuing %s", self.name, motion.hostname)
                self.inQueue.put(motion)
                count +=1
            
        return count

## Motion Class

Stuff relating to a single Motion device

In [11]:
HOST_STATUS_UNKNOWN = 0

HOST_STATUS_CONNECTED = 1
HOST_STATUS_INSPECTED = 2
HOST_STATUS_COMPLETED = 3

HOST_STATUS_REFUSED = -1
HOST_STATUS_TIMEOUT = -2
HOST_STATUS_GAIERROR = -3

DEFAULT_HTTP_PORT=80

DEFAULT_PING_TIMEOUT=5
DEFAULT_JSON_TIMEOUT=10

In [12]:
class Motion():
    '''Motion GPS'''
    
    def __init__(self, identifier, fileId):
        """Simple init method"""

        self.identifier = identifier
        self.fileId = fileId

        self.hostname = f'motion-{identifier}'

        self.addrInfo = None
        self.status = HOST_STATUS_UNKNOWN

        self.config = {}
        
        
    def getAddrInfo(self, port=DEFAULT_HTTP_PORT):
        """Get address information (IP address) for Motion"""

        try:
            addrInfo = socket.getaddrinfo(self.hostname, port, proto=socket.IPPROTO_TCP)[0][4]

            # Sky SR101 router will sometimes return 10.10.10.10 and should be treated the same as socket.gaierror
            if addrInfo[0] == '10.10.10.10':
                raise socket.gaierror

        # Only really expect a socket.gaierror exception, either from socket.getaddrinfo or because of 10.10.10.10
        except socket.gaierror:
            self.addrInfo = None

        # Just in case of some other (unexpected) issue!
        except Exception:
            self.addrInfo = None

        # The IP address of the Motion has been successfully identified. Yay!
        else:
            self.addrInfo = addrInfo
            
        return self.addrInfo


    def getStatus(self, timeout=DEFAULT_PING_TIMEOUT):
        """Check the status of the Motion.

        Note the use of self.addrInfo instead of using the hostname with s.connect((host, port)).

        This avoids erroneous 'gaierror: [Errno -5] No address associated with hostname' under certain circumstances."""

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()
        
        # Status can only be checked if address information is available
        if self.addrInfo:
            try:
                # Attempt connection to the standard HTTP socket as determined by socket.getaddrinfo()
                s = socket.socket()
                s.settimeout(timeout)
                s.connect(self.addrInfo)
                
                # Close the connection in a timely fashion - good practice
                s.shutdown(socket.SHUT_RDWR)
                s.close()
                
            # This should only happen if the IP address does not belong to a Motion
            except ConnectionRefusedError:
                self.status = HOST_STATUS_REFUSED

            # This should only happen when the Motion has been disconnected from the WiFi
            except socket.timeout:
                self.status = HOST_STATUS_TIMEOUT

            # Just in case of some other (unexpected) issue!
            except Exception:
                self.status = HOST_STATUS_UNKNOWN

            # The Motion is accepting connections. Yay!
            else:
                self.status = HOST_STATUS_CONNECTED

        # Address information is not known
        else:
            self.status = HOST_STATUS_GAIERROR

        return self.status
   

    def fetchJson(self, path, port=DEFAULT_HTTP_PORT, timeout=DEFAULT_JSON_TIMEOUT):
        """Fetch JSON from Motion"""

        result = None

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # JSON can only be fetched if address information is available
        if self.addrInfo:
            # Use addrInfo to avoid unnecessary DNS lookup which can take several seconds
            url = 'http://{}/{}'.format(self.addrInfo[0], path)

            # This is a simple but effective way to open a URL in Python
            response = urlopen(url, timeout=timeout)
            txt = response.read()

            # Parse the JSON and store it in a dictionary
            result = json.loads(txt)

        return result


    def summarise(self):
        """Print summary of device - legacy code, just for reference during development"""

        #print(self.hostname)
        #print()

        info = self.info

        #print('Username = {}'.format(info['motion']['username']))
        print('Identifier = {}'.format(info['motion']['identifier']))
        #print()

        hardware = info['motion']['hardware']
        firmware = info['motion']['firmware']

        #print('Hardware = {}'.format(hardware))
        print('Firmware = {}{}'.format(firmware, ' (latest)' if firmware == LATEST_FIRMWARE else ''))
        #print()

        # 'filesystem level' always appears to be 3

        level = info['motion']['battery']['level']
        charging = info['motion']['battery']['charging']

        print('Battery = {}%'.format(level))
        print('Battery is {}charging'.format('not ' if charging == False else ''))
        #print()

        settings = self.settings

        print('GNSS rate = {}'.format(settings['settings']['gnss_rate']))
        print()

        # Note: 'units': {'speed': 'kn', 'distance': 'nm'} does not impact logs (mm/s and mm)
        #        'start_date' is in UTC
        #        'timezone' and 'units' are applied by the built-in webserver
        #        'mode' always appears to be 0

        #logs = fetchJson(addresses[i][0], 'logs.json', timeout=20)
        #for log in reversed(logs['logs'][-1:]):
            #print(log)
        #print()

## Main Function

Startup and shutdown of all threads

In [13]:
def loadConfig():
    '''Read server config from JSON'''

    filename = os.path.join(projdir, CONFIG_DIR, 'server.json')
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        config = json.loads(jsonTxt)

    return config

In [14]:
def initLogging(level=logging.INFO, filename=None):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    # TODO - implement filename
    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)

In [15]:
def dateRange(startDate, endDate):
    '''Determine range of dates'''

    dates = []

    for i in range(int((endDate - startDate).days) + 1):
        dates.append(startDate + timedelta(days = i))
        
    return dates


def readMotions():
    '''Read Motions from CSV file'''
    
    motions = {}

    year = datetime.now().year
    csvPath = os.path.join(projdir, EVENTS_DIR, str(year), CONFIG_DIR, 'motions.csv')

    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for motionUser in csvReader:
            motionId = int(motionUser['Motion ID'])
            fileId = motionUser['File ID']

            startDate = datetime.strptime(motionUser['Start Date'], '%Y-%m-%d')
            endDate = datetime.strptime(motionUser['End Date'], '%Y-%m-%d')
            
            if motionId not in motions:
                motion = Motion(motionId, fileId=fileId)              
                motions[motionId] = motion
            else:
                motion = motions[motionId]

            for sessionDate in dateRange(startDate, endDate):
                sessionDate = sessionDate.strftime('%Y-%m-%d')

                # Note: Motion ID of zero (or negative) is a dummy record in the config
                if motionId > 0:
                    if sessionDate in motion.config:
                        logging.warning("Motion %i is assigned to multiple people on %s", motionId, sessionDate)
                    else:
                        motion.config[sessionDate] = motionUser
     
    return motions

In [16]:
def createThreadPools(config, motions, stopEvent):
    """Create the thread pools, including message queues"""

    threadPools = []

    # Detection   
    detectionInQueue = queue.Queue()
    detectionOutQueue = queue.Queue()
    detectionThreadPool = DetectionThreadPool(config, motions, stopEvent, detectionInQueue, detectionOutQueue)
    threadPools.append(detectionThreadPool)

    # Inspection    
    inspectionInQueue = detectionOutQueue
    inspectionOutQueue = queue.Queue()  
    inspectionThreadPool = InspectionThreadPool(config, motions, stopEvent, inspectionInQueue, inspectionOutQueue)
    threadPools.append(inspectionThreadPool)

    # Download
    downloadInQueue = inspectionOutQueue
    downloadOutQueue = queue.Queue()
    downloadThreadPool = DownloadThreadPool(config, motions, stopEvent, downloadInQueue, downloadOutQueue)
    threadPools.append(downloadThreadPool)

    # Disconnect
    disconnectionInQueue = queue.Queue()
    disconnectionOutQueue = queue.Queue()
    disconnectionThreadPool = DisconnectionThreadPool(config, motions, stopEvent, disconnectionInQueue, disconnectionOutQueue)
    threadPools.append(disconnectionThreadPool)

    return threadPools

In [17]:
def main():
    """Main function to start all of the threads"""

    # Read config
    config = loadConfig()

    # Initialise logging module
    initLogging(level=config['Logging']['Level'])
    
    # Read Motion data
    motions = readMotions()

    # Global event so that all threads can exit gracefully
    stopEvent = threading.Event()
    
    # Create the thread pools, including the message queues, etc.
    logging.debug("Creating thread pools")
    threadPools = createThreadPools(config, motions, stopEvent)

    # Start up all of the thread pools
    abort = True
    try:
        logging.info("Starting thread pools")

        for threadPool in threadPools:
            threadPool.start()

        logging.info("Thread pools started")

        abort = False
    
    except Exception:
        logging.error("Thread pools failed to start")

    # Just sleep until a keyboard interrupt occurs
    while not abort:
        try:
            time.sleep(10)
        
        # Intercept keyboard interrupts to ensure that all threads are properly stopped, prior to exit
        except KeyboardInterrupt:
            logging.info("Keyboard interrupt")
            abort = True

    # Stop threads gracefully
    logging.info("Stopping thread pools")
    stopEvent.set()
    for threadPool in threadPools:
        threadPool.join()
    logging.info("Thread pools stopped")


if __name__ == '__main__':
    main()

2022-10-08 11:54:52,INFO,Starting thread pools
2022-10-08 11:54:52,INFO,Thread pools started
2022-10-08 11:54:52,INFO,motion-601 detected and is accepting connections
2022-10-08 11:54:52,INFO,motion-615 detected and is accepting connections
2022-10-08 11:54:52,INFO,motion-618 detected and is accepting connections
2022-10-08 11:54:52,INFO,motion-625 detected and is accepting connections
2022-10-08 11:54:52,INFO,motion-601 battery level is 77%
2022-10-08 11:54:52,INFO,motion-615 battery level is 76%
2022-10-08 11:54:52,INFO,motion-617 detected and is accepting connections
2022-10-08 11:54:52,INFO,motion-618 battery level is 82%
2022-10-08 11:54:53,INFO,motion-625 battery level is 76%
2022-10-08 11:54:54,INFO,motion-617 battery level is 85%
2022-10-08 11:54:54,INFO,Keyboard interrupt
2022-10-08 11:54:54,INFO,Stopping thread pools
2022-10-08 11:54:55,INFO,Thread pools stopped
